#AG3 - Actividad Guiada 3
Nombre: Johnny Andres Illescas Fernandez <br>
Link:   https://colab.research.google.com/drive/1I2tflqeD5s9n6bI1aBT-H-7IONAtyWBB?usp=sharing <br>
Github: https://github.com/AndresGin/VIU_Algoritmos_de_optimizacion/tree/main/AG3
<br>




#Instalación de librerias

In [2]:
# Importación de la biblioteca requests
import requests

# Importación de urllib.request
import urllib.request

# Importación de la biblioteca tsplib95
import tsplib95

# Importación de otras dependencias que podrían ser útiles
import networkx as nx
import tabulate
import torch

In [3]:
import random   #Libreria para generar numeros y listas aleatorias
import copy     #Permite hacer copias de objetos en python: listas, diccionarios,...

#Carga de datos del problema

In [4]:
#Librerias y carga del problema

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion :
  # https://wwwproxy.iwr.uni-heidelberg.de/groups/comopt/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/usage.html
  # https://tsplib95.readthedocs.io/modules.html#module-tsplib95.models

#Matriz de distancias
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

#Objeto de tsplib95 para nuestro problema problema
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())
print(Aristas)

#Coordenadas(si estan disponibles en el ficher)
problem.get_display(1)

#Distancia
problem.get_weight(1, 2)



[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (0, 32), (0, 33), (0, 34), (0, 35), (0, 36), (0, 37), (0, 38), (0, 39), (0, 40), (0, 41), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (1, 33), (1, 34), (1, 35), (1, 36), (1, 37), (1, 38), (1, 39), (1, 40), (1, 41), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2,

34

#Funciones de la Actividad Guiada 3

In [5]:
#Funciones de la Actividad Guiada 3
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:] :
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)


#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

#Funciones Auxiliares

In [7]:
# Función para generar una población de soluciones aleatorias
def generar_poblacion(Nodos, N):
    poblacion = []  # Lista donde se almacenarán las soluciones

    for _ in range(N):
        # Crear una solución aleatoria (permutación aleatoria de los nodos)
        solucion = random.sample(Nodos, len(Nodos))  # Genera una permutación aleatoria de nodos
        poblacion.append(solucion)

    return poblacion

In [8]:
def Evaluar_Poblacion(poblacion, problem):
    mejor_solucion = None
    mejor_distancia = float('inf')  # Iniciar con un valor muy alto para encontrar la mínima

    # Evaluar cada individuo en la población
    for solucion in poblacion:
        distancia_total = 0

        # Calcular la distancia total del recorrido
        for i in range(len(solucion) - 1):
            distancia_total += problem.get_weight(solucion[i], solucion[i + 1])  # Distancia entre nodos consecutivos

        # También debemos sumar la distancia entre el último y el primer nodo (cerrar el ciclo)
        distancia_total += problem.get_weight(solucion[-1], solucion[0])

        # Si esta solución tiene una distancia menor, la actualizamos como mejor solución
        if distancia_total < mejor_distancia:
            mejor_distancia = distancia_total
            mejor_solucion = solucion

    return mejor_solucion, mejor_distancia

In [9]:
#Funcion de cruce. Recibe una poblacion(lista de soluciones) y devuelve la población ampliada con los hijos.
# Todos los individuos de la población son selecionados para el cruce(si la población es par)
# Podría aplicarse un proceso previo de selección para elegir los individuos que se desea cruzar.

# Función de cruce (crossover) que toma una población y genera hijos
def Cruzar(poblacion, mutacion, problem):
    hijos = []  # Lista de hijos generados
    poblacion_ampliada = []  # Población ampliada con los hijos

    # Selección de padres
    while len(hijos) < len(poblacion):  # Mientras no tengamos suficientes hijos
        # Seleccionar dos padres aleatorios
        padre1, padre2 = random.sample(poblacion, 2)

        # Realizar cruce (crossover de orden parcial - PMX)
        hijo1, hijo2 = crossover_pmx(padre1, padre2)

        # Opcional: Aplicar mutación a los hijos
        if random.random() < mutacion:
            hijo1 = mutar(hijo1)
        if random.random() < mutacion:
            hijo2 = mutar(hijo2)

        # Añadir los hijos a la lista de hijos
        hijos.append(hijo1)
        hijos.append(hijo2)

    # Añadir los hijos generados a la población ampliada
    poblacion_ampliada.extend(poblacion)
    poblacion_ampliada.extend(hijos)

    return poblacion_ampliada

# Función de cruce PMX (Crossover de Orden Parcial)
def crossover_pmx(padre1, padre2):
    # Elegir dos puntos de cruce aleatorios
    p1, p2 = sorted(random.sample(range(len(padre1)), 2))

    # Crear los hijos a partir de los padres
    hijo1 = [-1] * len(padre1)
    hijo2 = [-1] * len(padre2)

    # Copiar los segmentos entre los puntos de cruce
    hijo1[p1:p2+1] = padre1[p1:p2+1]
    hijo2[p1:p2+1] = padre2[p1:p2+1]

    # Rellenar los huecos en los hijos con los elementos que no están en el segmento
    rellenar_hijo(hijo1, padre2, p1, p2)
    rellenar_hijo(hijo2, padre1, p1, p2)

    return hijo1, hijo2

# Función para rellenar los huecos en los hijos con los nodos no repetidos
def rellenar_hijo(hijo, padre, p1, p2):
    for i in range(len(hijo)):
        if hijo[i] == -1:  # Si la posición está vacía
            # Rellenar con el nodo que no se encuentra en el segmento ya copiado
            nodo = padre[i]
            while nodo in hijo:  # Evitar nodos repetidos
                i_padre = padre.index(nodo)
                nodo = padre[i_padre]
            hijo[i] = nodo

# Función de mutación (cambio aleatorio en la permutación)
def mutar(solucion):
    # Cambiar dos posiciones aleatorias
    i, j = random.sample(range(len(solucion)), 2)
    solucion[i], solucion[j] = solucion[j], solucion[i]
    return solucion


In [10]:
#Funcion para generar hijos a partir de 2 padres:
# Se elige el metodo de 1-punto de corte pero es posible usar otros n-puntos, uniforme, dependiendo del problema

# Función para generar hijos a partir de 2 padres usando el cruce de 1-punto de corte
def Descendencia(padres, problem, mutacion):
    hijos = []  # Lista para almacenar los hijos generados

    # Seleccionar dos padres aleatorios de la lista de padres
    padre1, padre2 = random.sample(padres, 2)

    # Elegir un punto de corte aleatorio
    punto_corte = random.randint(1, len(padre1) - 1)

    # Crear los hijos mediante el cruce 1-punto
    hijo1 = padre1[:punto_corte] + padre2[punto_corte:]
    hijo2 = padre2[:punto_corte] + padre1[punto_corte:]

    # Aplicar mutación con cierta probabilidad
    if random.random() < mutacion:
        hijo1 = mutar(hijo1)  # Mutar el hijo1
    if random.random() < mutacion:
        hijo2 = mutar(hijo2)  # Mutar el hijo2

    # Añadir los hijos a la lista de descendencia
    hijos.append(hijo1)
    hijos.append(hijo2)

    return hijos

# Función de mutación (cambio aleatorio en la permutación)
def mutar(solucion):
    # Cambiar dos posiciones aleatorias
    i, j = random.sample(range(len(solucion)), 2)
    solucion[i], solucion[j] = solucion[j], solucion[i]
    return solucion

In [11]:
#Para el operador de cruce 1-punto los hijos generados no son soluciones(algunos nodos se repiten y otros no están)


def Factibilizar(solucion, problem):
    # Crear un conjunto con los nodos originales
    nodos_originales = set(problem.get_nodes())

    # Convertir la solución en un conjunto de nodos
    nodos_en_solucion = set(solucion)

    # Encontrar los nodos faltantes (que no están en la solución)
    nodos_faltantes = list(nodos_originales - nodos_en_solucion)

    # Encontrar los nodos repetidos (que aparecen más de una vez)
    nodos_repetidos = [nodo for nodo in solucion if solucion.count(nodo) > 1]

    # Reemplazar los nodos repetidos con los nodos faltantes
    for i in range(len(solucion)):
        if solucion.count(solucion[i]) > 1:
            if nodos_faltantes:
                solucion[i] = nodos_faltantes.pop()

    return solucion

In [12]:
#Funcion de mutación. Se eligen dos nodos y se intercambia. Se podrian añadir otros operaradores
# Se hace mutaciones mutacion% de las veces


# Función de mutación
def Mutar(solucion, mutacion):
    # Si la mutación debe ocurrir (según la probabilidad)
    if random.random() < mutacion:
        # Elegir dos nodos aleatorios para intercambiarlos
        i, j = random.sample(range(len(solucion)), 2)

        # Intercambiar los nodos seleccionados
        solucion[i], solucion[j] = solucion[j], solucion[i]

    return solucion

In [13]:
#Funcion de seleccion de la población. Recibe como parametro una poblacion y
# devuelve una poblacion a la que se ha eliminado individuos poco aptos(fitness alto) y para mantener una poblacion estable de N individuos
#Se tiene en cuenta el porcentaje elitismo pasado como parametro
# Para los individuos que no son de la elite podríamos usar una selección de ruleta(proporcional a su fitness)


# Función de selección de la población con elitismo y selección por ruleta
def Seleccionar(problem, poblacion, N, elitismo):
    # Evaluar la población (obtener fitness de cada individuo)
    fitness = [Evaluar_Individuo(individuo, problem) for individuo in poblacion]

    # Ordenar la población por fitness (de mayor a menor fitness)
    poblacion_ordenada = [x for _, x in sorted(zip(fitness, poblacion), reverse=True)]
    fitness_ordenado = sorted(fitness, reverse=True)

    # El número de individuos seleccionados para la nueva población
    num_elite = int(elitismo * N)  # El porcentaje de elitismo
    nueva_poblacion = poblacion_ordenada[:num_elite]  # Seleccionar a los mejores según elitismo

    # Selección por ruleta para los individuos que no están en la élite
    restantes = N - num_elite
    if restantes > 0:
        # Crear una ruleta proporcional a los fitness (fitness más alto, mayor probabilidad de selección)
        total_fitness = sum(fitness_ordenado[num_elite:])  # Fitness total de los individuos no élite
        probabilidades = [f / total_fitness for f in fitness_ordenado[num_elite:]]  # Probabilidades de selección

        # Seleccionar los individuos para completar la población
        seleccionados = random.choices(poblacion_ordenada[num_elite:], probabilities=probabilidades, k=restantes)

        # Añadir los seleccionados por ruleta a la nueva población
        nueva_poblacion.extend(seleccionados)

    return nueva_poblacion

# Función para evaluar el fitness de un individuo
def Evaluar_Individuo(individuo, problem):
    # Este es un ejemplo simple, se asume que el fitness es la inversa de la distancia total del recorrido
    # El fitness más alto corresponde al recorrido más corto (mínima distancia)
    total_distancia = 0
    for i in range(len(individuo) - 1):
        total_distancia += problem.get_weight(individuo[i], individuo[i + 1])

    # Cerrar el ciclo
    total_distancia += problem.get_weight(individuo[-1], individuo[0])

    # El fitness puede ser inverso de la distancia (cuanto menor la distancia, mayor el fitness)
    return 1 / total_distancia  # Fitness es la inversa de la distancia


#Proceso Principal

In [ ]:
#Funcion principal del algoritmo genetico
#######################################################3
def algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=100):
  # problem = datos del problema
  # N = Tamaño de la población
  # mutacion = probabilidad de una mutación
  # elitismo = porcion de la mejor poblacion a mantener
  # generaciones = nº de generaciones a generar para finalizar

  #Genera la poblacion inicial
  Nodos = list(problem.get_nodes())
  poblacion = generar_poblacion(Nodos,N)

  #Inicializamos valores para la mejor solucion
  (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)


  #Condicion de parada
  parar = False
  n=0
  #Inciamos el cliclo de generaciones
  while(parar == False) :

    #Cruce de la poblacion(incluye mutación)
    poblacion = Cruzar(poblacion, mutacion, problem)

    #Seleccionamos la población
    poblacion = Seleccionar(problem,poblacion, N, elitismo)

    #Evaluamos la nueva población
    (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)

    print("Generacion #", n, "\nLa mejor solución es:" , mejor_solucion, "\ncon distancia " , mejor_distancia, "\n")

    #Numero de generaciones. Criterio de parada
    if n==generaciones:
      parar = True
    n +=1

  return mejor_solucion


sol = algoritmo_genetico(problem=problem,N=500,mutacion=.3,elitismo=.40,generaciones=250)